In [ ]:
from data_classes.TextLightningDataModule import TextLightningDataModule
from models.ClassifierSystem import LightningClassifier
from data_classes.pretrained_embeddings import get_pretrained_embeddings
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_lightning.callbacks.gpu_stats_monitor import GPUStatsMonitor

In [ ]:
# Data and model settings
dataset = "Yelp"
num_class = 5
embedding = "Glove"
max_vectors = 10000
dim = 300


# Training settings
max_epochs=20
patience=3
lr=1e-3
batch_size=32
num_workers=0

# Log file:
log_file = "exp1"
log_file_csv = "exp1_csv"

In [ ]:
vocab, vectors = get_pretrained_embeddings(embedding =embedding, max_vectors = max_vectors, dim = dim)

In [ ]:
imdb_data = TextLightningDataModule(vocab, dataset=dataset, batch_size=batch_size, num_workers=num_workers)

In [ ]:
# logger_tensor = TensorBoardLogger(log_file, name='lstm')
# logger_csv = CSVLogger(log_file_csv, name="exp_csv")

In [ ]:
for model_type in ["RNN", "LSTM", "GRU"]:
    for output_layer_type in ["linear", "MLP"]:
        for num_layers in [1, 2]:
            for hidden_size in [100, 300]:
                # , logger=[logger_tensor, logger_csv]
                trainer = Trainer(fast_dev_run=True, max_epochs=max_epochs, gpus=1, auto_select_gpus=True, callbacks=[EarlyStopping(monitor="Val Loss", patience=3)])
                classifier = LightningClassifier(num_class=num_class, vocab=vocab, vectors=vectors, embedding_size=dim, learning_rate=lr, num_layers=num_layers, hidden_size=hidden_size, model_type=model_type, attention_type=None, output_layer_type=output_layer_type)
                trainer.fit(classifier, imdb_data)
                # trainer.test(classifier, imdb_data)